## Functions


#### Libraries

In [1]:
import numpy as np
from numpy import *
from numpy.linalg import inv
from commpy.modulation import QAMModem
import keras.backend as K
import tensorflow as tf

#### Functions

In [2]:
def Basic_Nonlinear_Distortion_Model(A, B, q, p, G, Vsat, IBO):
    val_IBO_m1dB = ((1 / sqrt(10 ** -0.1)) ** (2 * p) - 1) ** (1 / (2 * p)) * Vsat / (G)
    coeff_IBO_m1dB = val_IBO_m1dB * sqrt(10 ** (-IBO / 10))
    s = random.randn(1, 1000000)
    vin1 = sqrt(1 / 2) * (s + 1j * s)
    vin01 = coeff_IBO_m1dB * vin1
    a0 = absolute(vin01)
    a02 = a0 ** 2
    theta = angle(vin01)
    Am = (G * a0) / ((1 + (G * a0 / Vsat) ** (2 * p)) ** (1 / (2 * p)))
    Bm = (A * (a0 ** q)) / ((1 + (a0 / B) ** (q)))
    Sm = Am * exp(1j * (Bm))
    vout1 = Am * exp(1j * (theta + Bm))
    K0 = mean(vout1 * conj(vin01)) / mean(absolute(vin01) ** 2)
    sigma2_d = var(vout1 - K0 * vin01)
    return (K0, sigma2_d)


Basic_Nonlinear_Distortion_Model.__doc__ = """This function represents the basic nonlinear PA distortion model.
        In which we generate a random bit stream equivalent to the power amplifier input (Vin1) which is normalised by the Input Back-Off coefficients
        the considered PA is a SSPA modified Rapp model whose characteristic functions are given by the following formula:
        The AM-AM characterization represents the amplitude to amplitude conversion and can be represented as follows:     
        Am = (G * a0) / ((1 + (G * a0 / Vsat) ** (2 * p)) ** (1 / (2 * p))) 
        The AM-PM characterization represents the amplitude to phase conversion and can be represented as follows:
        Bm = (A * (a0 ** q)) / ((1 + (a0 / B) ** (q)))
        Where:  u is the magnitude of the input signal, G is small signal gain, Vsat is saturation level, p is the smoothness factor and A, B and q are ﬁtting parameters.
        a0 is the magnitude and theta is the phase of the input symbol
        Input: complex random symbols
        Output:
        -K0: Complex gain of PA
        -sigma2_d: Distortion vector of PA
"""

In [3]:
def hpa_sspa_modif_rapp(vin, Vsat, p, q, G, A, B):
    A = -345
    a0 = abs(vin)
    theta = np.angle(vin)
    Am = (G * a0) / ((1 + (G * a0 / Vsat) ** (2 * p)) ** (1 / (2 * p)))
    Bm = (A * (a0 ** q)) / ((1 + (a0 / B) ** (q)))
    vout = Am * np.exp(1j * (theta + Bm))
    return vout


hpa_sspa_modif_rapp.__doc__ = """ This function calculates the output of the power amplifier PA.
         The nonlinear behavior of PA can be described using the AM-AM compression and AM-PM conversion characteristics.
         Input:
         -vin: Signal to be amplified
         -Vsat: the saturation level
         -p: the smoothing factor
         -G: small signal gain
         -q, A and B: adjustment parameters
         Output: Amplified signal
        """

In [4]:
def find_K0_sigma2_d(vin, vout):
    K0 = np.mean(vout * np.conj(vin)) / np.mean(np.absolute(vin) ** 2)
    sigma2_d = np.var(vout - K0 * vin)
    return (K0, sigma2_d)


find_K0_sigma2_d.__doc__ = """This function provides the parameters of distortion of nonlinearity of PA.
         According to Bussgang's theorem, we can decompose the non-linear signal at the PA output into a linear function of the PA input and an uncorrelated distortion term.
         The amplified signal will be in this form ymt = qmtxmt + dm
         Input:
         -vin: Signal to be amplified
         -vout: Amplified signal
         Output:
         -K0: Complex gain of PA
         -sigma2_d: Distortion vector of PA
        """

In [5]:
def log10(x):
    numerator = tf.math.log(x)
    denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator


log10.__doc__ = """This function calculates the log10 of a tensor
        Input: a tensor
        Output: a tensor
       """

In [6]:
def NMSE(X_train, y_pred):
    NMS = K.zeros(shape=(1000))
    NMSEdb = K.zeros(shape=(1000))
    # recieveh=K.zeros([1000,10])
    y_predt = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    Y_gdr = y_predt[:1000, :100]
    Y_gdi = y_predt[:1000, 100:]
    Y_gd = tf.complex(Y_gdr, Y_gdi)
    X_testt = tf.convert_to_tensor(X_train, dtype=tf.float32)
    Shr = X_testt[:1000, :10]
    Shi = X_testt[:1000, 10:20]
    SSh = tf.complex(Shr, Shi)
    for i in range(1000):
        val_IBO_m1dB = (
            ((1 / np.sqrt(10 ** -0.1)) ** (2 * p) - 1) ** (1 / (2 * p)) * Vsat / (G)
        )
        coeff_IBO_m1dB = (
            val_IBO_m1dB
            * tf.math.sqrt((1 / K.var(Y_gd[i])))
            * np.sqrt(10 ** (-IBO / 10))
        )
        vin2 = coeff_IBO_m1dB * Y_gd[i]
        A = -345
        a0 = K.abs(vin2)
        theta = tf.math.angle(vin2)
        Am = (G * a0) / ((1 + (G * a0 / Vsat) ** (2 * p)) ** (1 / (2 * p)))
        Bm = (A * (a0 ** q)) / ((1 + (a0 / B) ** (q)))
        vout2 = tf.complex(Am, 0.0) * tf.math.exp(tf.complex(0.0, theta + Bm))
        Y_gd_amp = vout2 / coeff_IBO_m1dB
        Y_gd_amp0 = K.reshape(Y_gd_amp, (100, 1))
        HH = K.constant(H, dtype=tf.complex64)
        recieveh = K.dot(HH, (Y_gd_amp0))
        recievehh = K.reshape(recieveh, (1, 10))
        NMS = K.mean(K.abs(tf.math.subtract(recievehh, SSh[i]) ** 2)) / K.mean(
            K.abs(SSh[i]) ** 2
        )
        NMSEdb = 10 * log10(K.mean(NMS))
    return NMSEdb


NMSE.__doc__ = """This function is used to calculate the mean square error between the predicted symbols S and the symbols amplified then passed through the channel.
        This function represents a custom metric compiled by the compile () function of the Keras library.
        Input:
        -Y_true: A tensor which contains y_train
        -Y_Pred: A tensor which contains y_pred
        Output: A tensor which contains the mean square error in db.
       """

In [7]:
def NMSE_calcul(X_test, y_pred,p,q,G,A,B,Vsat,IBO,H):
    NMSE = np.zeros([1000, 1])
    recievehh = np.zeros([1000, 10], dtype=complex)
    Y_gdr = y_pred[:1000, :100]
    Y_gdi = 1j * y_pred[:1000, 100:]
    Y_gd = Y_gdr + Y_gdi
    Shr = X_test[:1000, :10]
    Shi = 1j * X_test[:1000, 10:20]
    SSh = Shr + Shi
    for i in range(1000):
        val_IBO_m1dB = (((1 / np.sqrt(10 ** -0.1)) ** (2 * p) - 1) ** (1 / (2 * p)) * Vsat / (G))
        coeff_IBO_m1dB = (val_IBO_m1dB * np.sqrt((1 / np.var(Y_gd[i]))) * np.sqrt(10 ** (-IBO / 10)))
        vin22 = coeff_IBO_m1dB * Y_gd[i]
        vout22 = hpa_sspa_modif_rapp(vin22, Vsat, p, q, G, A, B)
        Y_gd_amp = vout22 / coeff_IBO_m1dB
        recieveh = (H.dot(Y_gd_amp)).reshape((10))
        recievehh[i] = recieveh
        NMSE[i] = np.mean(np.abs(SSh[i] - recieveh) ** 2) / np.mean(np.abs(SSh[i]) ** 2)
    NMSEdb = 10 * np.log10(np.mean(NMSE))
    return NMSEdb


NMSE_calcul.__doc__ = """This function is used to calculate the mean square error between the predicted symbols S and the symbols amplified then passed through the channel.
        This function represents a custom metric compiled by the compile () function of the Keras library.
        Input:
        -Y_true:  y_test
        -Y_Pred:  y_pred
        Output:  the mean square error in db.
       """

In [8]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))


root_mean_squared_error.__doc__ = """
         This function calculates the square root of the mean of the squared errors.
         It is the most used metric for regression tasks.
         It is preferable in some cases, because the errors are first gridded before the average, which poses a high penalty on large errors.
         This implies that RMSE is useful when large errors are not desired.
         Between:
         -Y_true: A tensor which contains y_train
         -Y_Pred: A tensor which contains y_pred
         Output: A tensor which contains the mean square error.

        """